In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, Activation,Flatten
from keras.layers import Embedding, Input, Dense, Dropout, Lambda, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [0]:
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’"/|_#$%ˆ&*˜‘+=<>()[]{} '


Embedding and net design Inspired from *Very Deep Convolutional Networks for Text Classification* (https://arxiv.org/abs/1606.01781).

In [0]:
char_dict = {}
max_len=1024
dataset_path='/content/gdrive/My Drive/ag_data/'
import csv
import numpy as np

for i,c in enumerate(alphabet):
  char_dict[c] = i+1


def char2vec(text):
		data = np.zeros(max_len)
		for i in range(0, len(text)):
			if i > max_len:
				return data
			elif text[i] in char_dict:
				data[i] = char_dict[text[i]]
			else:
				# unknown character set to be 68
				data[i] = 68
		return data

def load_csv_file(filename, num_classes):
		"""
		Load CSV file, generate one-hot labels and process text data as Paper did.
		"""
		all_data = []
		labels = []
		with open(filename) as f:
			reader = csv.DictReader(f, fieldnames=['class'], restkey='fields')
			for row in reader:
				# One-hot
				one_hot = np.zeros(num_classes)
				one_hot[int(row['class']) - 1] = 1
				labels.append(one_hot)
				# Char2vec
				data = np.ones(max_len)*68
				text = row['fields'][-1].lower()
				all_data.append(char2vec(text))
		f.close()
		return np.array(all_data), np.array(labels)
  
  
train_data, train_label = load_csv_file(dataset_path+'train.csv', 4)
test_data, test_label = load_csv_file(dataset_path+'test.csv', 4)


In [0]:
val_data=test_data[:3800]
val_label=test_label[:3800]

test_data=test_data[3800:]
test_lable=test_label[3800:]

In [0]:
def just_dense(num_classes, depth=9, sequence_length=1024, embedding_dim=16, 
          shortcut=False, pool_type='max', sorted=True, use_bias=False, input_tensor=None):

    inputs = Input(shape=(sequence_length, ), name='inputs')
    embedded_chars = Embedding(input_dim=sequence_length, output_dim=embedding_dim)(inputs)
    out = BatchNormalization()(embedded_chars)
    out=Flatten()(out)
    # Dense Layers
    out = Dense(16, activation='relu')(out)
    out=Dropout(rate=0.25)(out)
    out = Dense(16, activation='relu')(out)
    out=Dropout(rate=0.25)(out)
    out = Dense(16, activation='relu')(out)
    out=Dropout(rate=0.25)(out)
    out = Dense(num_classes, activation='softmax')(out)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = inputs

    # Create model.
    model = Model(inputs=inputs, outputs=out, name='VDCNN')
    return model

In [0]:
import keras
class NBatchLogger(keras.callbacks.Callback):
    """
    A Logger that log average performance per `display` steps.
    """
    def __init__(self, display):
        self.step = 0
        self.display = display
        self.metric_cache = {}

    def on_batch_end(self, batch, logs={}):
        self.step += 1
        for k in self.params['metrics']:
            if k in logs:
                self.metric_cache[k] = self.metric_cache.get(k, 0) + logs[k]
        if self.step % self.display == 0:
            metrics_log = ''
            for (k, v) in self.metric_cache.items():
                val = v / self.display
                if abs(val) > 1e-3:
                    metrics_log += ' - %s: %.4f' % (k, val)
                else:
                    metrics_log += ' - %s: %.4e' % (k, val)
            print('step: {}/{} ... {}'.format(self.step,
                                          self.params['steps'],
                                          metrics_log))
            self.metric_cache.clear()

out_batch = NBatchLogger(display=1000)

In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 60:
        lrate = 0.000125
    elif epoch > 40:
        lrate = 0.00025
    elif epoch > 20:
        lrate = 0.0005
    elif epoch > 10:
        lrate = 0.00075           
    return lrate
from keras.callbacks import LearningRateScheduler
lrate = LearningRateScheduler(lr_schedule,verbose=0)

In [0]:
from keras.callbacks import LearningRateScheduler
lrate = LearningRateScheduler(lr_schedule,verbose=0)

In [37]:
model = just_dense(num_classes=train_label.shape[1])
opt=keras.optimizers.Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint("/content/gdrive/My Drive/checkpoints/"+"weights.best_small_dense_3.hdf5", verbose=1, monitor='val_acc',save_best_only=True, mode='auto')  
out_batch = NBatchLogger(display=10000)
model.fit(train_data, train_label,batch_size=32,epochs=100,callbacks=[checkpoint,lrate],validation_data=(val_data,val_label),shuffle=True,verbose=0)



Epoch 00001: val_acc improved from -inf to 0.36737, saving model to /content/gdrive/My Drive/checkpoints/weights.best_small_dense_3.hdf5

Epoch 00002: val_acc improved from 0.36737 to 0.41395, saving model to /content/gdrive/My Drive/checkpoints/weights.best_small_dense_3.hdf5

Epoch 00003: val_acc improved from 0.41395 to 0.43342, saving model to /content/gdrive/My Drive/checkpoints/weights.best_small_dense_3.hdf5

Epoch 00004: val_acc did not improve from 0.43342

Epoch 00005: val_acc improved from 0.43342 to 0.45368, saving model to /content/gdrive/My Drive/checkpoints/weights.best_small_dense_3.hdf5

Epoch 00006: val_acc did not improve from 0.45368

Epoch 00007: val_acc improved from 0.45368 to 0.47895, saving model to /content/gdrive/My Drive/checkpoints/weights.best_small_dense_3.hdf5

Epoch 00008: val_acc did not improve from 0.47895

Epoch 00009: val_acc did not improve from 0.47895

Epoch 00010: val_acc did not improve from 0.47895


KeyboardInterrupt: ignored

# Another Try

I think that I did that embedding stuff wrong from above. Backtracking and using the Keras library to do the heavy lifting.

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

In [0]:
import pandas as pd
dataset_path='/content/gdrive/My Drive/ag_data/'

import numpy as np

In [0]:
train=pd.read_csv(dataset_path+'train.csv',header=None)

In [0]:
train[1]=train[1].astype(str)
train[2]=train[2].astype(str)

In [11]:
train[1].values

array(['Wall St. Bears Claw Back Into the Black (Reuters)',
       'Carlyle Looks Toward Commercial Aerospace (Reuters)',
       "Oil and Economy Cloud Stocks' Outlook (Reuters)", ...,
       'Saban not going to Dolphins yet', "Today's NFL games",
       'Nets get Carter from Raptors'], dtype=object)

In [0]:
new_col=[]
for i in range(len(train)):
  new_col.append(train[1].values[i]+" "+train[2].values[i])

^used both the title and the text for classification. Idea given by Joseph Bentivegna.

In [0]:
docs=np.array(new_col)

In [14]:
labels=train[0].values
labels

array([3, 3, 3, ..., 2, 2, 2])

In [0]:
# integer encode the documents
vocab_size = 50000
encoded_docs = [one_hot(d, vocab_size) for d in docs]

In [0]:
one_hot_labels=np.zeros((len(train),4))
row_num=0
for i in labels:
  one_hot_labels[row_num,i-1]=1
  row_num+=1

In [0]:
max_length = 1024
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [0]:
train=pd.read_csv(dataset_path+'test.csv',header=None)
new_col=[]
for i in range(len(train)):
  new_col.append(train[1].values[i]+" "+train[2].values[i])
docs=np.array(new_col)
labels=train[0].values
encoded_docs = [one_hot(d, vocab_size) for d in docs]
test_one_hot_labels=np.zeros((len(train),4))
row_num=0
for i in labels:
  test_one_hot_labels[row_num,i-1]=1
  row_num+=1
max_length = 1024
test_padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [0]:
val_padded_docs=test_padded_docs[3800:]
val_one_hot_labels=test_one_hot_labels[3800:]

test_padded_docs=test_padded_docs[:3800]
test_one_hot_labels=test_one_hot_labels[:3800]

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, 512, input_length=max_length))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
checkpoint = ModelCheckpoint("/content/gdrive/My Drive/checkpoints/"+"weights.best_high_space.hdf5", verbose=1, monitor='val_acc',save_best_only=True, mode='auto')  
# fit the model
model.fit(padded_docs, one_hot_labels, epochs=5,validation_data=(val_padded_docs,val_one_hot_labels), verbose=0,callbacks=[checkpoint],shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1024, 512)         25600000  
_________________________________________________________________
flatten_3 (Flatten)          (None, 524288)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 2097156   
Total params: 27,697,156
Trainable params: 27,697,156
Non-trainable params: 0
_________________________________________________________________
None

Epoch 00001: val_acc improved from -inf to 0.91263, saving model to /content/gdrive/My Drive/checkpoints/weights.best_high_space.hdf5

Epoch 00002: val_acc did not improve from 0.91263

Epoch 00003: val_acc did not improve from 0.91263

Epoch 00004: val_acc did not improve from 0.91263

Epoch 00005: val_acc did not improve from 0.91263


In [0]:
Embedding?

## Validation

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, 512, input_length=max_length))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))
model.load_weights("/content/gdrive/My Drive/checkpoints/"+"weights.best_high_space.hdf5")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])



In [30]:
model.evaluate(val_padded_docs,val_one_hot_labels)

3800/3800 [==============================] - 1s 201us/step


[0.27163526183680486, 0.9126315790728519]

# Test 

In [31]:
model.evaluate(test_padded_docs,test_one_hot_labels)

3800/3800 [==============================] - 1s 172us/step


[0.33820385878023346, 0.901578947368421]

# Discussion

I sort of fell into the solution on its own. Below is an explanation for parameter choices.

I knew that tutorials like [this](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/) were obviously meant for shorter sentences with much less unique terms than full news articles and titles, so the hashmap needed to be larger. Thus the 50,000 dimension in the one hot of the documents.

The maximum length term is given by the max size of a sentence. Obviously the tutorial here has a maximum length that is way too small, thus the 1024 length. The training set maximum length is actually around 980, but I bumped it up just in case.

The last parameter is the output dimension of the Embeddding. I chose 512, just because it was larger and I felt as though it was necessary. I could make this smaller, I hear other students could run this on much smaller output dimensions, but I do believe in the following philosophy:

# If it ain't broke, don't fix it.